In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam


import random
import numpy as np

import re
import nltk
import string
import numpy as np
from torch.utils.data import Dataset
from torch import nn
from tqdm import tqdm

import matplotlib.pyplot as plt



In [ ]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_excel(r'/content/drive/MyDrive/SoftCom/IoT_Security_Dataset.xlsx')
df = df.drop(['SentenceId', 'PostId', 'Run'], axis=1)
df.head()

,Sentence,Security
0,UCSR2C = (1 << UMSEL21) | (1 << UMSEL20); // |...,0
1,There's only a warning in the eclipse logcat :...,0
2,"When I place an RFID tag in front of the RFID,...",0
3,?j\nWhy?,0
4,"Edit:\nI had set the digital pin 0 to output, ...",0


In [ ]:
df_ml= pd.read_excel(r'/content/drive/MyDrive/SoftCom/IoT_ML_Dataset.xlsx')
df_ml = df_ml.drop(['SentenceId','PostId', 'PostType', 'KeywordML'], axis=1)
df_ml.head()

,Sentence,HasML
0,I'm trying to calculate a neural network with ...,1
1,Currently I'm trying to replicate this wonderf...,1
2,I'd like to feed the scanned images into a dee...,1
3,"I have experience with machine learning, but m...",1
4,Edit: For fear of sounding like my question is...,1


In [ ]:
df['Security'].value_counts()

0    4870
1    1049
Name: Security, dtype: int64

In [ ]:
df_ml['HasML'].value_counts()

0    2399
1     801
Name: HasML, dtype: int64

In [ ]:
train_df, test_df = train_test_split(df, stratify=df['Security'], test_size=0.2, random_state=42)

train_df, val_df = train_test_split(train_df, stratify=train_df['Security'], test_size=0.2, random_state=42)




print("\n*************** TRAIN DATA *****************")
display(train_df)

print("\n*************** VAL DATA *****************")
display(val_df)

print("\n*************** TEST DATA *****************")
display(test_df)



*************** TRAIN DATA *****************


,Sentence,Security
3028,It should be possible to achieve latencies on ...,0
1135,My goal is to activate an electric lock that w...,0
3332,See this link for how to do it.,0
3950,"The concept of any object being ephemeral, for...",0
1316,Is JTAG the normal method (I think that is wha...,0
...,...,...
3928,an implementation of the interpreter in the fi...,0
5036,If you are logged out when your Mac is lost bu...,1
982,It is possible that this issue\nis resolved by...,0
3406,This worked for me on Lubuntu 18.04 VM on my M...,0



*************** VAL DATA *****************


,Sentence,Security
1018,I have to restart Visual Studio at all first.e...,0
3179,I always encode the authentication header - \n...,1
3444,Try changing the minimum API to 18.,0
1644,I'm a new programmer for devices,0
1102,Heres what it looks like now:\nchar Pieces[27]...,0
...,...,...
3064,Now I know I have counted off all the digits i...,0
5136,"For example, Gagawa is perfect when a develope...",0
163,Today I installed Arch Linux onto my Raspberry...,0
379,"(I get ""3"" on the console)\nThe Arduino IDE on...",0



*************** TEST DATA *****************


,Sentence,Security
5508,Dancer is designed to be as effortless as poss...,0
4425,SAS code written with SAS Macro will be compil...,0
5672,WebSphere DataPower Integration Appliance XI52...,1
5124,"WinBUGS, on the other hand, is an established ...",0
2166,"@Barmar is correct, there is nothing that stop...",0
...,...,...
4113,It is a family of RDBMS Oracle Drivers for Qt.,0
2433,The array indexing operator never does bit-lev...,0
1491,"What is this error, how can I fix it?",0
3350,Hope this helps!,0


In [ ]:
train_df_ml, test_df_ml = train_test_split(df_ml, stratify=df_ml['HasML'], test_size=0.2, random_state=42, shuffle=True)

train_df_ml, val_df_ml = train_test_split(train_df_ml, stratify=train_df_ml['HasML'], test_size=0.2, random_state=42, shuffle=True)

In [ ]:
print(train_df_ml['HasML'].value_counts())
print(val_df_ml['HasML'].value_counts())
print(test_df_ml['HasML'].value_counts())

0    1535
1     513
Name: HasML, dtype: int64
0    384
1    128
Name: HasML, dtype: int64
0    480
1    160
Name: HasML, dtype: int64


In [ ]:
test_df['Security'].value_counts()

0    974
1    210
Name: Security, dtype: int64

In [ ]:
#row = train_df.index[train_df['Sentence'] == 'If the LTPA token is not yet present the user must authenticate normally.'][0]
#row

4796

In [ ]:
#train_df.to_excel(r'/content/drive/MyDrive/SoftCom/IoT_Security_Dataset_train.xlsx', index=False)

In [ ]:
train_df.iloc[3761]

Sentence    If the LTPA token is not yet present the user ...
Security                                                    1
Name: 4796, dtype: object

In [ ]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3788 entries, 3028 to 943
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  3788 non-null   object
 1   Security  3788 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 88.8+ KB


In [ ]:
def reset_random_state():
    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

reset_random_state()


In [ ]:
texts = train_df.Sentence.values.tolist()

In [ ]:
texts[3761]

'If the LTPA token is not yet present the user must authenticate normally.'

In [ ]:
example = 3761

In [ ]:
class securityDataset(Dataset):
    def __init__(self, dataframe, tokenizer):



        texts = dataframe.Sentence.values.tolist()

        #if( len(texts) == 3788):
        #  print(texts[example])

        texts = np.array([np.array(val) for val in texts])



        texts = [self._preprocess(text) for text in texts]

        #if( len(texts) == 3788):
        #  print('processed:')
        #  print(texts[example])

        self._print_random_samples(texts)

        self.texts = [tokenizer(text, padding='max_length',
                                max_length=150,
                                truncation=True,
                                return_tensors="pt")
                      for text in texts]

        #if( len(texts) == 3788):
        #  print('tokenized:')
        #  print(texts[example])

        print('after tokenize')

        if 'Security' in dataframe:
            classes = dataframe.Security.values.tolist()
            self.labels = classes

    def _print_random_samples(self, texts):
        np.random.seed(42)
        random_entries = np.random.randint(0, len(texts), 5)

        for i in random_entries:
            print(f"Entry {i}: {texts[i]}")

        print()

    def _preprocess(self, text):

        text = self._remove_multiple_spaces(text)

        text = self._lowercase(text)
        text = self._remove_punctuation(text)
        text = self._remove_numbers(text)

        text_tokens = self._tokenize(text)
        text_tokens = self._stopword_filtering(text_tokens)
        text_tokens = self._stemming(text_tokens)
        text = self._stitch_text_tokens_together(text_tokens)

        return text.strip()



    def _remove_multiple_spaces(self, text):
        return re.sub(r'\s+', ' ', text)


    def _stitch_text_tokens_together(self, text_tokens):
        return " ".join(text_tokens)

    def _tokenize(self, text):
        return nltk.word_tokenize(text, language="english")

    def _stopword_filtering(self, text_tokens):
        stop_words = nltk.corpus.stopwords.words('english')

        return [token for token in text_tokens if token not in stop_words]

    def _stemming(self, text_tokens):
        porter = nltk.stem.porter.PorterStemmer()
        return [porter.stem(token) for token in text_tokens]

    def _remove_numbers(self, text):
        return re.sub(r'\d+', ' ', text)

    def _lowercase(self, text):
        return text.lower()

    def _remove_punctuation(self, text):
        return ''.join(character for character in text if character not in string.punctuation)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        label = -1
        if hasattr(self, 'labels'):
            label = self.labels[idx]

        return text, label


In [ ]:
test_df_ml

,SentenceId,Sentence,HasML
800,670621,I would like to introduce myself to StackOverf...,1
1193,393,How can I make this arduino accept my program?,0
1131,331,If I don't have it in a loop then whatever set...,0
3046,2246,How do I get a signal out from the Qt applicat...,0
1491,691,I've started sending characters until I reache...,0
...,...,...,...
1940,1140,Here is what the emitting code looks like: _x...,0
1576,776,I've coded it in C++ using three classes and t...,0
78,246909,UPDATE:_x000D_\nSo I've been able to combine t...,1
981,181,"So, just to test, I pluged a wire in the Analo...",0


In [ ]:
class MLDataset(Dataset):
    def __init__(self, dataframe, tokenizer):



        texts = dataframe.Sentence.values.tolist()

        #if( len(texts) == 3788):
        #  print(texts[example])

        texts = np.array([np.array(val) for val in texts])



        texts = [self._preprocess(text) for text in texts]

        #if( len(texts) == 3788):
        #  print('processed:')
        #  print(texts[example])

        self._print_random_samples(texts)

        self.texts = [tokenizer(text, padding='max_length',
                                max_length=150,
                                truncation=True,
                                return_tensors="pt")
                      for text in texts]

        #if( len(texts) == 3788):
        #  print('tokenized:')
        #  print(texts[example])

        print('after tokenize')

        if 'HasML' in dataframe:
            classes = dataframe.HasML.values.tolist()
            self.labels = classes

    def _print_random_samples(self, texts):
        np.random.seed(42)
        random_entries = np.random.randint(0, len(texts), 5)

        for i in random_entries:
            print(f"Entry {i}: {texts[i]}")

        print()

    def _preprocess(self, text):

        text = self._remove_multiple_spaces(text)

        text = self._lowercase(text)
        text = self._remove_punctuation(text)
        text = self._remove_numbers(text)

        text_tokens = self._tokenize(text)
        text_tokens = self._stopword_filtering(text_tokens)
        text_tokens = self._stemming(text_tokens)
        text = self._stitch_text_tokens_together(text_tokens)

        return text.strip()



    def _remove_multiple_spaces(self, text):
        return re.sub(r'\s+', ' ', text)


    def _stitch_text_tokens_together(self, text_tokens):
        return " ".join(text_tokens)

    def _tokenize(self, text):
        return nltk.word_tokenize(text, language="english")

    def _stopword_filtering(self, text_tokens):
        stop_words = nltk.corpus.stopwords.words('english')

        return [token for token in text_tokens if token not in stop_words]

    def _stemming(self, text_tokens):
        porter = nltk.stem.porter.PorterStemmer()
        return [porter.stem(token) for token in text_tokens]

    def _remove_numbers(self, text):
        return re.sub(r'\d+', ' ', text)

    def _lowercase(self, text):
        return text.lower()

    def _remove_punctuation(self, text):
        return ''.join(character for character in text if character not in string.punctuation)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        label = -1
        if hasattr(self, 'labels'):
            label = self.labels[idx]

        return text, label


In [ ]:
class IoTClassifier(nn.Module):
    def __init__(self, base_model):
        super().__init__()

        self.bert = base_model
        self.fc1 = nn.Linear(768, 32)
        self.fc2 = nn.Linear(32, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0][:, 0]
        x = self.fc1(bert_out)
        x = self.relu(x)


        x = self.fc2(x)
        x = self.sigmoid(x)

        return x


In [ ]:
def train(model, train_dataloader, val_dataloader, learning_rate, epochs):
    best_val_loss = 99999999999999
    early_stopping_threshold_count = 0


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    model = model.to(device)
    criterion = criterion.to(device)

    for epoch in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_input, train_label in tqdm(train_dataloader):
            attention_mask = train_input['attention_mask'].to(device)
            input_ids = train_input['input_ids'].squeeze(1).to(device)

            train_label = train_label.to(device)

            output = model(input_ids, attention_mask)

            loss = criterion(output, train_label.float().unsqueeze(1))

            total_loss_train += loss.item()

            acc = ((output >= 0.5).int() == train_label.unsqueeze(1)).sum().item()
            total_acc_train += acc

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        with torch.no_grad():
            total_acc_val = 0
            total_loss_val = 0

            model.eval()

            for val_input, val_label in tqdm(val_dataloader):
                attention_mask = val_input['attention_mask'].to(device)
                input_ids = val_input['input_ids'].squeeze(1).to(device)

                val_label = val_label.to(device)

                output = model(input_ids, attention_mask)

                loss = criterion(output, val_label.float().unsqueeze(1))

                total_loss_val += loss.item()

                acc = ((output >= 0.5).int() == val_label.unsqueeze(1)).sum().item()
                total_acc_val += acc

            print(f'Epochs: {epoch + 1} '
                  f'| Train Loss: {total_loss_train / len(train_dataloader): .3f} '
                  f'| Train Accuracy: {total_acc_train / (len(train_dataloader.dataset)): .3f} '
                  f'| Val Loss: {total_loss_val / len(val_dataloader): .3f} '
                  f'| Val Accuracy: {total_acc_val / len(val_dataloader.dataset): .3f}')

            if best_val_loss > total_loss_val:
                best_val_loss = total_loss_val
                torch.save(model, f"best_model.pt")
                print("Saved model")
                early_stopping_threshold_count = 0
            else:
                early_stopping_threshold_count += 1

            if early_stopping_threshold_count >= 1:
                print("Early stopping")
                break


In [ ]:
reset_random_state()

#BERT_MODEL = "roberta-base"
#BERT_MODEL = "bert-base-uncased"
BERT_MODEL = "xlnet-base-cased"
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

base_model = AutoModel.from_pretrained(BERT_MODEL)



In [ ]:
s=tokenizer('ltpa token yet present user must authent normal', padding='max_length', max_length=150, truncation=True, return_tensors="pt")
s

In [ ]:
train_dataloader = DataLoader(securityDataset(train_df, tokenizer), batch_size=8, shuffle=True, num_workers=0)
val_dataloader = DataLoader(securityDataset(val_df, tokenizer), batch_size=8, num_workers=0)

model = IoTClassifier(base_model)

Entry 3174: anorm object relat mapper
Entry 3507: ttl argument repres number second function call
Entry 860: convers queri hold sch lock block concurr transact attempt acquir schm lock
Entry 1294: thank sorri bad english
Entry 1130: primarili tool develop especi web develop work open sourc project

after tokenize
Entry 102: littl exampl toggl led includ stdioh includ stdlibh includ sysmmanh includ fcntlh defin handleerrormsg perrormsg exitexitfailur defin piodbankd xa defin piooffset xfffff execut board long sz sysconfscpages printfldnrsz
Entry 435: still access file browser text file doesnt chang
Entry 860: fastmm default memori manag delphi win win bit applic
Entry 270: toolbar easili customis use twitter bootstrap icon provid flexabl around toolbar display number icon
Entry 106: eeglab interact matlab toolbox process continu eventrel eeg meg electrophysiolog data incorpor independ compon analysi ica timefrequ analysi artifact reject eventrel statist sever use mode visual averag sing

In [ ]:
train_dataloader_ml = DataLoader(MLDataset(train_df_ml, tokenizer), batch_size=8, shuffle=True, num_workers=0)
val_dataloader_ml = DataLoader(MLDataset(val_df_ml, tokenizer), batch_size=8, num_workers=0)

model = IoTClassifier(base_model)

Entry 1126: rnn abbrevi recurr neural network class artifici neural net feedback
Entry 1459: im attempt setup aw greengrass mxnet machin learn raspberri pi
Entry 860: output isx chang speed x motor write x motor read x x case x bad rover host invok updatespeedvaluesgood invok returnspeedvalu
Entry 1294: want write program spit data frame specif length byte valu rang
Entry 1130: problem havent got experi ethernet good idea start look

after tokenize
Entry 102: project led sensor array conect arduino board
Entry 435: total time secondsx that consol seem output well rather frustrat
Entry 348: arduino ide first wrote includ neuralnetworkh problem compil sure call gener function matlab coder
Entry 270: consid whether question might better suit cross valid stackexchang site statist machin learn data analysi
Entry 106: host send charact byte data

after tokenize


In [ ]:
reset_random_state()

learning_rate = 1e-5
#learning_rate = 2e-5
epochs = 2
train(model, train_dataloader, val_dataloader, learning_rate, epochs)

In [ ]:
reset_random_state()

learning_rate = 1e-5
epochs = 2
train(model, train_dataloader_ml, val_dataloader_ml, learning_rate, epochs)

100%|██████████| 64/64 [00:04<00:00, 14.54it/s]


Epochs: 1 | Train Loss:  0.344 | Train Accuracy:  0.868 | Val Loss:  0.067 | Val Accuracy:  0.988
Saved model


100%|██████████| 64/64 [00:04<00:00, 14.24it/s]


Epochs: 2 | Train Loss:  0.095 | Train Accuracy:  0.975 | Val Loss:  0.041 | Val Accuracy:  0.992
Saved model


In [ ]:
def get_text_predictions(model, loader):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = model.to(device)


    results_predictions = []
    results_probs = []
    with torch.no_grad():
        model.eval()
        for data_input, _ in tqdm(loader):
            attention_mask = data_input['attention_mask'].to(device)
            input_ids = data_input['input_ids'].squeeze(1).to(device)


            output = model(input_ids, attention_mask)
            results_probs.append(output)

            output = (output > 0.5).int()
            results_predictions.append(output)

    return torch.cat(results_predictions).cpu().detach().numpy(), torch.cat(results_probs).cpu().detach().numpy()


In [ ]:
model = torch.load("best_model.pt")


In [ ]:
train_dataloader = DataLoader(securityDataset(train_df, tokenizer), batch_size=8, shuffle=False, num_workers=0)
val_dataloader = DataLoader(securityDataset(val_df, tokenizer), batch_size=8, shuffle=False, num_workers=0)
test_dataloader = DataLoader(securityDataset(test_df, tokenizer), batch_size=8, shuffle=False, num_workers=0)

In [ ]:
train_dataloader_ml = DataLoader(securityDataset(train_df_ml, tokenizer), batch_size=8, shuffle=False, num_workers=0)
val_dataloader_ml = DataLoader(securityDataset(val_df_ml, tokenizer), batch_size=8, shuffle=False, num_workers=0)
test_dataloader_ml = DataLoader(securityDataset(test_df_ml, tokenizer), batch_size=8, shuffle=False, num_workers=0)

Entry 1126: rnn abbrevi recurr neural network class artifici neural net feedback
Entry 1459: im attempt setup aw greengrass mxnet machin learn raspberri pi
Entry 860: output isx chang speed x motor write x motor read x x case x bad rover host invok updatespeedvaluesgood invok returnspeedvalu
Entry 1294: want write program spit data frame specif length byte valu rang
Entry 1130: problem havent got experi ethernet good idea start look

after tokenize
Entry 102: project led sensor array conect arduino board
Entry 435: total time secondsx that consol seem output well rather frustrat
Entry 348: arduino ide first wrote includ neuralnetworkh problem compil sure call gener function matlab coder
Entry 270: consid whether question might better suit cross valid stackexchang site statist machin learn data analysi
Entry 106: host send charact byte data

after tokenize
Entry 102: h inmemori platform distribut scalabl machin learn
Entry 435: edit dohx wasnt great pipelin improv betterx globl armtest 

In [ ]:
train_df["text_prediction_probs"] = get_text_predictions(model, train_dataloader)[1]
val_df["text_prediction_probs"] = get_text_predictions(model, val_dataloader)[1]
test_df["text_prediction_probs"] = get_text_predictions(model, test_dataloader)[1]


100%|██████████| 148/148 [00:10<00:00, 14.21it/s]


In [ ]:
train_df_ml["text_prediction_probs"] = get_text_predictions(model, train_dataloader_ml)[1]
val_df_ml["text_prediction_probs"] = get_text_predictions(model, val_dataloader_ml)[1]
test_df_ml["text_prediction_probs"] = get_text_predictions(model, test_dataloader_ml)[1]


100%|██████████| 80/80 [00:05<00:00, 15.00it/s]


In [ ]:
train_df.iloc[example]

Sentence                 If the LTPA token is not yet present the user ...
Security                                                                 1
text_prediction_probs                                             0.943163
Name: 4796, dtype: object

In [ ]:
test_df['pred'] = test_df['text_prediction_probs'].apply(lambda x: 0 if x <0.5 else 1)
test_df

,Sentence,Security,text_prediction_probs,pred
5508,Dancer is designed to be as effortless as poss...,0,0.009770,0
4425,SAS code written with SAS Macro will be compil...,0,0.010034,0
5672,WebSphere DataPower Integration Appliance XI52...,1,0.986832,1
5124,"WinBUGS, on the other hand, is an established ...",0,0.009740,0
2166,"@Barmar is correct, there is nothing that stop...",0,0.009969,0
...,...,...,...,...
4113,It is a family of RDBMS Oracle Drivers for Qt.,0,0.010076,0
2433,The array indexing operator never does bit-lev...,0,0.009936,0
1491,"What is this error, how can I fix it?",0,0.010474,0
3350,Hope this helps!,0,0.010508,0


In [ ]:
test_df_ml['pred'] = test_df_ml['text_prediction_probs'].apply(lambda x: 0 if x <0.5 else 1)
test_df_ml

,Sentence,HasML,text_prediction_probs,pred
800,I would like to introduce myself to StackOverf...,1,0.990215,1
1193,How can I make this arduino accept my program?,0,0.009481,0
1131,If I don't have it in a loop then whatever set...,0,0.009344,0
3046,How do I get a signal out from the Qt applicat...,0,0.009379,0
1491,I've started sending characters until I reache...,0,0.009144,0
...,...,...,...,...
1940,Here is what the emitting code looks like: _x...,0,0.009329,0
1576,I've coded it in C++ using three classes and t...,0,0.009172,0
78,UPDATE:_x000D_\nSo I've been able to combine t...,1,0.989749,1
981,"So, just to test, I pluged a wire in the Analo...",0,0.009178,0


In [ ]:
confusion_matrix = pd.crosstab(test_df['Security'], test_df['pred'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)


Predicted    0    1
Actual             
0          966    8
1           15  195


In [ ]:
confusion_matrix = pd.crosstab(test_df_ml['HasML'], test_df_ml['pred'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)


Predicted    0    1
Actual             
0          476    4
1            2  158
